In [0]:
#Installing dependencies for RDKit
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!time bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!time conda install -q -y -c conda-forge rdkit

--2019-12-14 07:20:32--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

PREFIX=/usr/local
Unpacking payload ...
Solving environment: - \ | / - \ | / - \ | / - \ | / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.2.0=py37_0
    - ca-certificates==2019.10.16=0
    - certifi==2019.9.11=py37_0
    - cffi==1.13.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.0=py37h7b6447c_0
    - conda==4.7.12=py37_0
    - cryptography==2.8=py37h1ba5d50_0
    - idna==2.8=py37_0
    - libedit==3.1.20181209=h

In [0]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
root = "/content/drive/My Drive/dsb2/"

In [0]:
import os

file_list = os.listdir(root)

num_mols = len(file_list)
print(num_mols)

13000


In [0]:
def read_xyz(file_name):
    with open(file_name, 'rb') as file:
        num_atoms = int(file.readline())
        properties = file.readline().split()[5:17] # only take the properties used in the experiments 
        properties = [num.replace(b'*^', b'e') for num in properties] 
        properties = [float(prop) for prop in properties]
        atom_types = [0]*num_atoms
        coords = np.array(np.zeros([num_atoms,3]))
        for na in range(num_atoms):
            coord_line = file.readline().split()
            atom_types[na] = coord_line[0]
            xyz_coords = coord_line[1:4]
            xyz_coords = [num.replace(b'*^', b'e') for num in xyz_coords] 
            coords[na,:] = [float(num) for num in xyz_coords]  
        vib_freqs = file.readline()
        smiles = file.readline().split()[0]
        inchis = file.readline()
        
    return smiles, properties, atom_types, coords

In [0]:
import concurrent.futures
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import as_completed

In [0]:
import multiprocessing as mp
import numpy as np
from multiprocessing.pool import ThreadPool
import time


start = time.time()
N= mp.cpu_count()
files = os.scandir(root)
print (N)
with mp.pool.ThreadPool(processes = 8) as p:
        results= p.map(read_xyz, [root+file.name for file in files])
      
end = time.time()
print(end - start)

2
19.752538919448853


In [0]:
print(len(results))

13000


In [0]:
def column(matrix, i):
    return [row[i] for row in matrix]

smiles =column(results,0)
properties =column(results,1)
atom_types =column(results,2)

In [0]:
import pandas as pd
import numpy as np

In [0]:
dtype = [('Col1','int32'), ('Col2','float32'), ('Col3','float32'),('Col4','int32'), ('Col5','float32'), ('Col6','float32'),('Col7','int32'), ('Col8','float32'), ('Col9','float32'),('Col10','int32'), ('Col11','float32'), ('Col12','float32'),('Col13','float32'), ('Col14','float32'),('Col15','float32'), ('Col16','float32'),('Col17','float32')]
values = properties
index = ['Row'+str(i) for i in range(1, len(values)+1)]

df = pd.DataFrame(values, index=index)

df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
Row1,0.3676,68.74,-0.2352,0.0065,0.2417,814.7537,0.142229,-363.839405,-363.831976,-363.831032,-363.870471,29.280
Row2,1.8866,64.52,-0.2485,-0.0075,0.2409,794.2784,0.129457,-383.721976,-383.714630,-383.713685,-383.753023,28.544
Row3,1.7780,73.30,-0.2542,0.0838,0.3380,883.2429,0.177401,-349.011613,-349.003832,-349.002888,-349.042983,30.929
Row4,1.2074,73.61,-0.2583,0.0729,0.3312,881.9664,0.177890,-349.011971,-349.004251,-349.003307,-349.043189,31.339
Row5,2.1403,66.86,-0.2510,0.0868,0.3378,831.9533,0.152815,-384.933448,-384.925779,-384.924835,-384.964883,29.600


In [0]:
len(df)

13000

In [0]:
# normalize target values to have a mean of 0 and std of 1
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() 
data_scaled=scaler.fit_transform(df)
df = pd.DataFrame(data_scaled , index=index)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
Row1,-1.497174,0.376456,0.381924,-0.143865,-0.337901,-0.570023,0.300064,-0.077239,-0.077249,-0.077249,-0.077207,0.189786
Row2,-0.487877,-0.094090,-0.147182,-0.438433,-0.354346,-0.652968,-0.079868,-0.534428,-0.534445,-0.534445,-0.534387,0.027847
Row3,-0.560036,0.884913,-0.373942,1.482575,1.641695,-0.292575,1.346334,0.263719,0.263720,0.263720,0.263736,0.552609
Row4,-0.939170,0.919479,-0.537050,1.253233,1.501910,-0.297746,1.360880,0.263710,0.263710,0.263710,0.263732,0.642819
Row5,-0.319307,0.166829,-0.246638,1.545697,1.637584,-0.500348,0.614968,-0.562285,-0.562295,-0.562295,-0.562253,0.260194


In [0]:
import numpy as np

zinc_charset = [
    ' ', '#', ')', '(', '+', '-', '/', '1', '3', '2', '5', '4', '7', '6', '8',
    '=', '@', 'C', 'B', 'F', 'I', 'H', 'O', 'N', 'S', '[', ']', '\\', 'c', 'l',
    'o', 'n', 'p', 's', 'r'
]


class OneHotFeaturizer():
  """
  NOTE(LESWING) Not Thread Safe in initialization of charset
  """

  def __init__(self, charset=zinc_charset, padlength=60):
    """
    Parameters
    ----------
    charset: obj:`list` of obj:`str`
      Each string is length 1
    padlength: int
      length to pad the smile strings to
    """
    self.charset = charset
    self.pad_length = padlength

  def featurize(self, mols, verbose=True, log_every_n=1000):
    """
    Parameters
    ----------
    mols: obj
      List of rdkit Molecule Objects
    verbose: bool
      How much logging
    log_every_n:
      How often to log
    Returns
    -------
    obj
      numpy array of features
    """
    from rdkit import Chem
    smiles = [Chem.MolToSmiles(mol) for mol in mols]
    if self.charset is None:
      self.charset = self._create_charset(smiles)
    return np.array([self.one_hot_encoded(smile) for smile in smiles])

  def one_hot_array(self, i):
    """
    Create a one hot array with bit i set to 1
    Parameters
    ----------
    i: int
      bit to set to 1
    Returns
    -------
    obj:`list` of obj:`int`
      length len(self.charset)
    """
    return [int(x) for x in [ix == i for ix in range(len(self.charset))]]

  def one_hot_index(self, c):
    """
    TODO(LESWING) replace with map lookup vs linear scan
    Parameters
    ----------
    c
      character whose index we want
    Returns
    -------
    int
      index of c in self.charset
    """
    return self.charset.index(c)

  def pad_smile(self, smile):
    """
    Pad A Smile String to self.pad_length
    Parameters
    ----------
    smile: str
    Returns
    -------
    str
      smile string space padded to self.pad_length
    """

    return smile.ljust(self.pad_length)

  def one_hot_encoded(self, smile):
    """
    One Hot Encode an entire SMILE string
    Parameters
    ----------
    smile: str
      smile string to encode
    Returns
    -------
    object
      np.array of one hot encoded arrays for each character in smile
    """
    return np.array([
        self.one_hot_array(self.one_hot_index(x)) for x in self.pad_smile(smile)
    ])

  def untransform(self, z):
    """
    Convert from one hot representation back to SMILE
    Parameters
    ----------
    z: obj:`list`
      list of one hot encoded features
    Returns
    -------
    Smile Strings picking MAX for each one hot encoded array
    """
    z1 = []
    for i in range(len(z)):
      s = ""
      for j in range(len(z[i])):
        oh = np.argmax(z[i][j])
        s += self.charset[oh]
      z1.append([s.strip()])
    return z1

  def _create_charset(self, smiles):
    """
    create the charset from smiles
    Parameters
    ----------
    smiles: obj:`list` of obj:`str`
      list of smile strings
    Returns
    -------
    obj:`list` of obj:`str`
      List of length one strings that are characters in smiles.  No duplicates
    """
    s = set()
    for smile in smiles:
      for c in smile:
        s.add(c)
    return [' '] + sorted(list(s))

In [0]:
from rdkit import Chem

In [0]:
 mols = [Chem.MolFromSmiles(smile) for smile in smiles]

In [0]:
featurizer = OneHotFeaturizer()

In [0]:
biggest_mol_size = max([len(atom_list) for atom_list in atom_types])

In [0]:
biggest_mol_size

26

In [0]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt


%matplotlib inline

torch.manual_seed(2) 


In [0]:
len(mols)

13000

In [0]:
featurizer

In [0]:
one_hots = featurizer.featurize(mols)

In [0]:
one_hots.shape

(13000, 60, 35)

In [0]:
y_mu=df[df.columns[0]]
y=y_mu
y_mu=torch.FloatTensor(y_mu)
y=y.to_numpy()


In [0]:
y.shape

(13000,)

In [0]:
m=y.reshape((13000,1)) 

In [0]:
class Data(object):
  def __init__(self, x=None,y=None):
        self.x = x
        self.y = y

In [0]:
input_list=[]
for i in range(13000):
  data = Data(x=torch.FloatTensor(one_hots[i]),y=torch.FloatTensor(m[i]))
  input_list.append(data)

In [0]:
input_list2=[]
for i in range(13000):
 inputs = (torch.LongTensor(one_hots[i]),torch.LongTensor(m[i]))
 input_list2.append(inputs)

In [0]:
input_list2=[]
for i in range(13000):
  data = [np.array(one_hots[i],dtype='float64'),np.array(m[i],dtype='float64')]
  input_list2.append(data)



In [0]:
x = torch.cat(input_list2).view(len(input_list2), 1, -1)
x


In [0]:
dataset=input_list2

In [0]:
# a simple custom collate function, just to show the idea
def my_collate(batch):
    data = [item[0] for item in batch]
    target = [item[1] for item in batch]
    return torch.FloatTensor([data, target])

In [0]:
from torch.utils.data import Dataset, DataLoader
# Split datasets.
test_dataset = dataset[:960]
val_dataset = dataset[960:1920]
train_dataset =dataset[1920:12000]
test_loader = DataLoader(test_dataset, batch_size=60, shuffle=False, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=60, shuffle=False, num_workers=4)
train_loader = DataLoader(train_dataset, batch_size=60, shuffle=True, num_workers=8)

In [0]:
input_dim = 35
hidden_dim = 60

In [0]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModel, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim

        # Number of hidden layers
        self.layer_dim = layer_dim

        # Building your LSTM
        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, feature_dim)
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)
    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim,dtype=torch.float64).requires_grad_()

        # Initialize cell state
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim,dtype=torch.float64).requires_grad_()

        # 28 time steps
        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

        # Index hidden state of last time step
        # out.size() --> 100, 28, 100
        # out[:, -1, :] --> 100, 100 --> just want last time step hidden states! 
        out = self.fc(out[:, -1, :]) 
        # out.size() --> 100, 10
        return out

In [0]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [0]:
model = LSTMModel(input_dim=35, hidden_dim=60, layer_dim=60, output_dim=1)
model=model.double()
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
                                                       factor=0.7, patience=5,
                                                       min_lr=0.00001)

In [0]:
import torch.nn.functional as F

In [0]:
# Train the model


def train(epoc):
    itera=0
    model.train()
    loss_all=0
    for data, target in train_loader:
        #data, target=data.to(device), target.to(device)
        #data, target=data.cuda(), target.cuda()
        outputs=model(data)
        loss = F.mse_loss(outputs,target)
        #print('Iter: {:03d}, Loss: {:.7f}'.format(itera, lr, loss)) 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_all+=loss.item()
        itera=itera+1
    return loss_all / len(train_loader.dataset)


def test(loader):
    model.eval()
# Test the model
#with torch.no_grad():
    error = 0
    total = 0
    for data, target in loader:
         #data, target=data.to(device), target.to(device)
         #data, target=data.cuda(), target.cuda()
         outputs=model(data)
         error+=(outputs-target).abs().sum().item()
    return error / len(loader.dataset)


best_val_error = None
for epoch in range(1, 301):
    lr = scheduler.optimizer.param_groups[0]['lr']
    loss = train(epoch)
    val_error = test(val_loader)
    scheduler.step(val_error)

    if best_val_error is None or val_error <= best_val_error:
        test_error = test(test_loader)
        best_val_error = val_error

    print('Epoch: {:03d}, LR: {:7f}, Loss: {:.7f}, Validation MAE: {:.7f}, '
          'Test MAE: {:.7f}'.format(epoch, lr, loss, val_error, test_error))    
    #print('Test Accuracy of the model on the test smiles : {} %'.format(100 * correct / total)) 

# Save the model checkpoint
#torch.save(model.state_dict(), 'model.ckpt')

Epoch: 001, LR: 0.001000, Loss: 1.0392415, Validation MAE: 0.7307579, Test MAE: 0.7586130
Epoch: 002, LR: 0.001000, Loss: 1.0390482, Validation MAE: 0.7312262, Test MAE: 0.7586130
Epoch: 003, LR: 0.001000, Loss: 1.0391751, Validation MAE: 0.7308318, Test MAE: 0.7586130
Epoch: 004, LR: 0.001000, Loss: 1.0388677, Validation MAE: 0.7304512, Test MAE: 0.7563305
